In [ ]:
import pandas as pd
import numpy as np
import cupy as cp
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import math
import re
import nltk
from nltk.tokenize import word_tokenize
import glob
import json

import gensim
from gensim.models import Doc2Vec, Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
import warnings
warnings.filterwarnings('ignore')


In [ ]:
!nvcc --version

In [ ]:
!pip install cupy-cuda101

In [ ]:
priorart_PATH = "/content/gdrive/My Drive/GQP/Data/df_priorart.csv"
priorart = pd.read_csv(priorart_PATH)
priorart['noun'] = data['noun'].astype('str')
appeal_PATH = "/content/gdrive/My Drive/GQP/Data/df_appeal.csv"
appeal = pd.read_csv(appeal_PATH)
appeal['noun'] = data['noun'].astype('str')

In [ ]:
df_appeal.head()

In [ ]:
Reversed = df_appeal[df_appeal['decision']=='Reversed']
Affirmed = df_appeal[df_appeal['decision']=='Affirmed']

In [ ]:
Reversed.head()

# draft

In [ ]:
wordDict = set()
data['length'] = 0
for i in range(data.shape[0]):
  if i % 100000 == 0:
    print(i)
  data['length'][i]= len(data['noun'][i])
  words = data['noun'][i].split(' ')
  for w in words:
    wordDict.add(w)

In [ ]:
data.head()

In [ ]:
data[data.length < 10].shape

In [ ]:
data = data[data['length'] >= 10] 
data.shape

In [ ]:
def computeTF(wordDict, bagOfWords): 
    word_count = dict.fromkeys(wordDict, 0)
    for word in bagOfWords:
      word_count[word] += 1
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in word_count.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [ ]:
len(data.index), data.shape[0]

In [ ]:
# path = "/content/gdrive/My Drive/"
# file = open(path+'words.csv', 'w')
# file.write(', '.join(wordDict)+'\n')

for i in tqdm(data.index):  
  di = data['noun'][i].split(' ')
  di_tf = computeTF(wordDict, di)
  line = []
  for k in di_tf:
    line.append(str(di_tf[k]))
  break
  # file.write(','.join(line)+'\n')  
# file.close()

In [ ]:
arr = np.array(list(di_tf.values()))
arr[arr!=0].shape

In [ ]:
a = np.zeros(10,10)
a

In [ ]:
TF = np.zeros((data.shape[0], len(wordDict)))

In [ ]:
a = []
for k in di_tf:
  a.append(di_tf[k])
s = ','.join([str(n) for n in a])
print(len(s.split(',')))
print(len([float(n) for n in s.split(',') if float(n)>0]))

In [ ]:
def computeIDF(documents):    
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

# TF-Idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['noun'])
feature_names = vectorizer.get_feature_names()

In [ ]:
vectors.shape

In [ ]:
len(feature_names)

In [ ]:
vectorizer_appeal = TfidfVectorizer()
vectors_appeal = vectorizer_appeal.fit_transform(df_appeal['noun'])
feature_names_appeal = vectorizer.get_feature_names()

In [ ]:
vectors_appeal.shape

# Word2vec

In [ ]:
sentences = []
for i in tqdm(range(priorart.shape[0])):
  sentences.append(priorart['noun'][i].split(' '))
len(sentences)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
model = Word2Vec(min_count = 1,  size = 100, window = 5, sg= 1, workers = cores)
# model1 = gensim.models.Word2Vec(data['noun'], ) 
model.build_vocab(sentences)

In [ ]:
Vocab = model.wv.vocab
len(Vocab)

In [ ]:
# from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1

In [ ]:
model.train(sentences, total_examples=model.corpus_count, epochs=20, compute_loss=True, callbacks=[callback()])

# Cosine similarty

In [ ]:
new_word = []
for w in tqdm(feature_names_appeal):
  if w not in feature_names:
    new_word.append(w)

In [ ]:
vectors_appeal[Reversed.index[0]][:10]

In [ ]:
Reversed.shape[0] + Affirmed.shape[0]

In [ ]:
priorart

# classification

In [ ]:
Reversed['new'] = 0
# Reversed['new_nouns'] = 0
new_nouns = [] #[id, number, list]
for i in tqdm(Reversed.index):
  sentence = Reversed['noun'][i].split(' ')
  count = 0
  new = []
  for w in sentence:
    if w not in feature_names:
      count +=1
      new.append(w)
  if count > 0:
    new_nouns.append([i, count, new])
  Reversed['new'][i] = count
  # Reversed['new_nouns'][i] = new

In [ ]:
Reversed[Reversed.new>0]

In [ ]:
Affirmed['new'] = 0
new_nouns_affirmed = [] #[id, number, list]
for i in tqdm(Affirmed.index):
  sentence = Affirmed['noun'][i].split(' ')
  count = 0
  new = []
  for w in sentence:
    if w not in feature_names:
      count +=1
      new.append(w)
  if count > 0:
    new_nouns_affirmed.append([i, count, new])
  Affirmed['new'][i] = count  

In [ ]:
Affirmed.head()